In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sys import path

In [19]:
import requests
import pandas as pd
from os import getcwd
import urllib3
from bs4 import BeautifulSoup
from typing import List
from tqdm import tqdm
from time import sleep
import datetime as dt
from random import randint
from time import strptime, mktime, time

import numpy as np
from glob import glob

In [4]:
PROJECT_PATH = getcwd()[:getcwd().find("notebooks")][:-1]
path.append(f"{PROJECT_PATH}\\src")

In [37]:
from chrome_driver import (
    open_driver, 
    load_url, 
    get_driver_page_source)
from utils import dump_pickle, load_pickle

In [6]:
PROJECT_NAME = "wsj_tweet_scrapping"
DATA_PATH = f"{PROJECT_PATH}\\data"
EXCEL_PATH = f"{DATA_PATH}\\excel"
PICKLE_PATH = f"{DATA_PATH}\\pickle"

In [7]:
def extract_text_from_url(url):
    """Extract text from the URL."""
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text)
    http = urllib3.PoolManager()
    response = http.request(method="GET", url=url)
    soup = BeautifulSoup(response.data)
    complete_texts = ""
    for text in soup.find_all(name="p"):
        complete_texts += " " + text.text
    return complete_texts

In [8]:
def clean_text(text):
    """Clean input text."""
    discard_texts = ["\n", "\t", "\r", "\xa0"] 
    for word in discard_texts: 
        text = text.replace(word, " ")
    text = " ".join(text.split())
    return text

In [9]:
def extract_urls_from_tweet_source(text_soup : str):
    soup = BeautifulSoup(text_soup)
    urls = []
    for url in soup.find_all(name="a", attrs={"data-url":True}):
        url = url["data-url"]
        if "wsj" in url:
            urls.append(url)
    return urls

In [10]:
dates = pd.date_range(start=dt.date(2009,1,1), end=dt.date(2020,9,1), freq=pd.offsets.MonthBegin(1))
dates = dates.sort_values(ascending=False)

In [15]:
# processing raw tweets
processed_tweets = []

t = tqdm(range(len(dates)))
for i in t:
    year = dates[i].year
    month = dates[i].month
    t.set_description(f"Iter : {i}, year : {year}, month : {month}")
    
    # reading raw tweets
    file_name = f"twint_wsj_logistics_tweets_{year}_{str(month).rjust(2,'0')}"
    monthly_tweets = pd.read_excel(f"{EXCEL_PATH}\\{file_name}.xlsx")
    
    if  len(monthly_tweets) == 0:
        continue
        
  
    for i in range(len(monthly_tweets)):
        tweet = {}
        tweet["data-item-id"] = monthly_tweets["data-item-id"].iloc[i]
        tweet["data-conversation-id"] = monthly_tweets["data-conversation-id"].iloc[i]
        tweet["date"] = monthly_tweets["date"].iloc[i]
        tweet["tweet"] = monthly_tweets["tweet"].iloc[i]
        tweet["url"] = []
        for url in extract_urls_from_tweet_source(monthly_tweets["all-data"].iloc[i]):
            tweet["url"].append(url)
        processed_tweets.append(tweet)

Iter : 140, year : 2009, month : 1: 100%|████████████████████████████████████████████| 141/141 [01:57<00:00,  1.20it/s]


In [16]:
processed_tweets_df = pd.DataFrame(processed_tweets)
cols_check = [col for col in processed_tweets_df if not isinstance(processed_tweets_df[col].iloc[0], list)]
processed_tweets_df = processed_tweets_df.drop_duplicates(subset=cols_check).reset_index(drop = True)

processed_tweets_df["num_urls"] = processed_tweets_df["url"].apply(lambda x: len(x))
processed_tweets_df = processed_tweets_df[processed_tweets_df["num_urls"] > 0].reset_index(drop=True)
processed_tweets_df.drop("num_urls", axis=1, inplace=True)

In [17]:
processed_tweets_df

,data-item-id,data-conversation-id,date,tweet,url
0,1309123013057622016,1309123013057622016,2020-09-24,Today’s newsletter - Cleaning Vehicle Emissi...,[https://on.wsj.com/2EuJDdb]
1,1306223859855753218,1306223859855753218,2020-09-16,FedEx's Christmas in July; Probing Nikola Cl...,[https://on.wsj.com/2Fsewzz]
2,1305500577133203456,1305500577133203456,2020-09-14,Shipping’s E-Commerce Drive; Short-Selling N...,[https://on.wsj.com/3iv2qDW]
3,1304230721146380290,1304230721146380290,2020-09-10,Brexit’s New Alarms; Railroad Offloads Bidde...,[https://on.wsj.com/3inRU1m]
4,1308758522033823750,1308758522033823750,2020-09-23,Airlines Turn to Freight; Driving Robot Truc...,[https://on.wsj.com/32TvO1a]
...,...,...,...,...,...
2473,108461828237037568,108461828237037568,2011-08-30,Sina Acquires 9% Stake in Video Company Tudo...,[http://on.wsj.com/rffYru]
2474,108419013956222976,108419013956222976,2011-08-29,BofA Cashes Its China Chips on.wsj.com/oKfhAb\n,[http://on.wsj.com/oKfhAb]
2475,108232952936271872,108232952936271872,2011-08-29,"S&P Downgrades Sino-Forest, Withdraws Rating...",[http://on.wsj.com/pzI1nC]
2476,108224377857912832,108224377857912832,2011-08-29,Cathay to Offer Premium Economy on.wsj.com/p...,[http://on.wsj.com/p3q984]


In [18]:
# getting total number of unique urls
total_urls = []
for urls in processed_tweets_df["url"]:
    for url in urls:
        total_urls.append(url)
total_urls = np.unique(total_urls)

In [26]:
## incase of already extracted news
try: 
    news_pickles = []
    news_pickles = [pickle for pickle in glob(pathname=f"{PICKLE_PATH}//*.pickle") if "wsj_logistics_news"]
    news = load_pickle(max(news_pickles))
except:
    news = {}
print(f"Found {len(news)} news")

Found 2472 news


In [38]:
driver = open_driver()

In [21]:
news = {}
url_extract_success = []
url_extract_fail = []

In [49]:
for url in tqdm(total_urls):
    if url not in url_extract_success and url not in url_extract_fail:
        try:
            load_url(driver=driver, url=url) 
            sleep(randint(2,5))
            soup = get_driver_page_source(driver=driver) 
            complete_texts = ""
            for text in soup.find_all(name="p"):
                complete_texts += " " + text.text
            news[url] = complete_texts
            url_extract_success.append(url)
        except Exception as e:
            url_extract_fail.append(url)
            print(f"Failed extracting news, exception e : {e}")

 57%|███████████████████████████████████████████▋                                | 1420/2472 [25:42<7:37:08, 26.07s/it]

Failed loading page, exception : Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: MicrosoftEdge=86.0.622.51)



100%|████████████████████████████████████████████████████████████████████████████| 2472/2472 [2:04:35<00:00,  3.02s/it]


In [50]:
pickle_name = f"wsj_logistics_news_{int(time())}"
dump_pickle(obj=news, filename=f"{PICKLE_PATH}//{pickle_name}")

## Second Attempt to Download

In [42]:
url_extract_fail = [url for url in total_urls if len(news[url]) < 200]
url_extract_success_attempt2 = []

In [44]:
for url in tqdm(url_extract_fail):
    if url not in url_extract_success_attempt2:
        load_url(driver=driver, url=url) 
        sleep(randint(2,5))
        soup = get_driver_page_source(driver=driver) 
        complete_texts = ""
        for text in soup.find_all(name="p"):
            complete_texts += " " + text.text
        news[url] = complete_texts
    else:
        url_extract_success_attempt2.append(url)

100%|████████████████████████████████████████████████████████████████████████████████| 167/167 [15:14<00:00,  5.48s/it]


# Preparing final news dataframe

In [96]:
news_df = []
for i in tqdm(range(len(processed_tweets_df))):
    values = processed_tweets_df.values[i]
    row = {}
    row["date"] = values[2]
    row["tweet"] = values[3]
    for url in values[4]:
        row["url"] = url
        if url in news.keys():
            row["news"] = news[url]
        else:
            row["news"] = ""
        news_df.append(row)
        
news_df = pd.DataFrame(news_df)
news_df["news"] = news_df["news"].astype(str)
news_df["date"] = pd.to_datetime(news_df["date"])
news_df = news_df.drop_duplicates().reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████| 2478/2478 [00:01<00:00, 2227.09it/s]


In [103]:
from_year = min(news_df["date"].dt.year)
to_year = max(news_df["date"].dt.year)
news_df.to_excel(f"{EXCEL_PATH}\\wsj_logistics_news_{from_year}-{to_year}.xlsx", 
                 index=False)

In [ ]:
# t = tqdm(range(len(dates)))
# for i in t:
#     year = dates[i].year
#     month = dates[i].month
#     t.set_description(f"Iter : {i}, year : {year}, month : {month}")
    
#     # reading raw tweets
#     file_name = f"twint_wsj_logistics_tweets_{year}_{str(month).rjust(2,'0')}"
#     monthly_tweets = pd.read_excel(f"{EXCEL_PATH}\\{file_name}.xlsx")
    
#     if  len(monthly_tweets) == 0:
#         continue
        
#     # processing raw tweets
#     processed_tweets = []
#     for i in range(len(monthly_tweets)):
#         tweet = {}
#         tweet["data-item-id"] = monthly_tweets["data-item-id"].iloc[i]
#         tweet["data-conversation-id"] = monthly_tweets["data-conversation-id"].iloc[i]
#         tweet["date"] = monthly_tweets["date"].iloc[i]
#         tweet["tweet"] = monthly_tweets["tweet"].iloc[i]
#         for url in extract_urls_from_tweet_source(monthly_tweets["all-data"].iloc[i]):
#             tweet["url"] = url
#             processed_tweets.append(tweet)
#     processed_tweets = pd.DataFrame(processed_tweets)
#     processed_tweets = processed_tweets.drop_duplicates().reset_index(drop = True)
# #     print(f"After processing : {len(processed_tweets)} entries")
    
#     if len(processed_tweets) == 0:
#         continue
        
#     # fetching news articles
#     monthly_texts = []
#     for url in tqdm(processed_tweets["url"].values):
#         try:
#             text = extract_text_from_url(url)
#             text = clean_text(text)
#             text = {"url": url, "text": text}
#             monthly_texts.append(text) 
#             sleep(2)
#         except Exception as e:
#             failed_downloading_urls.append([year, month, url])
#             print(f"failed extracting news, exception : {e}")
            
    
#     # saving news article
#     monthly_texts = pd.DataFrame(monthly_texts) 
#     monthly_texts = pd.merge(processed_tweets, monthly_texts, on="url", how="left")
# #     print(f"After extracting texts : {len(monthly_texts)} entries")
    
#     file_name = f"twint_wsj_logistics_news_{year}_{str(month).rjust(2,'0')}"
#     monthly_texts.drop_duplicates().to_excel(f"{EXCEL_PATH}\\{file_name}.xlsx", index=False)

In [12]:
# pd.DataFrame(failed_downloading_urls, columns=["year", "month", "url"]).to_excel(f"{EXCEL_PATH}\\failed_download_news_url-{int(time())}.xlsx",
#                                                                                  index=False) 